# MÓDULO 8: INTRODUCCIÓN A LANGGRAPH
## SESIÓN 1: Ejercicios Prácticos - Construyendo Agentes

Este notebook contiene 10 ejercicios progresivos para construir la arquitectura cognitiva de agentes:
- Diseñar la memoria de corto plazo (Estado)
- Crear capacidades específicas (Nodos)
- Conectar el flujo de decisiones (Edges y Routers)
- Implementar razonamiento iterativo (Ciclos)

**Instrucciones:** Completa cada ejercicio en orden. Cada ejercicio construye las habilidades necesarias para el siguiente.

Importamos las dependencias necesarias para todos los ejercicios.

In [1]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END

---
## EJERCICIO 1: Definir la Memoria de Corto Plazo del Agente

Crea una clase `EstadoTareas` usando `TypedDict` con los siguientes campos:
- `tareas`: Lista de strings con las tareas pendientes
- `tarea_actual`: String con la tarea en proceso
- `completadas`: Entero con el número de tareas completadas
- `estado`: String que puede ser "activo" o "inactivo"

In [2]:
# TODO: Define la clase EstadoTareas usando TypedDict

# Tu código aquí


# Prueba: crea una instancia del estado
estado_prueba = {
    "tareas": ["Comprar pan", "Hacer ejercicio"],
    "tarea_actual": "",
    "completadas": 0,
    "estado": "activo"
}

print(f"Estado creado: {estado_prueba}")

Estado creado: {'tareas': ['Comprar pan', 'Hacer ejercicio'], 'tarea_actual': '', 'completadas': 0, 'estado': 'activo'}


---

## EJERCICIO 2: Crear el Estado y un Nodo de Transformación

Implementa tanto la estructura del estado como la lógica del nodo `nodo_transformar` que realice lo siguiente:

1. **Definir el Estado:** Crea una clase `EstadoTexto(TypedDict)` con los campos `texto_entrada` (str) y `texto_salida` (str).
2. **Implementar el Nodo:** Crea la función `nodo_transformar` que:
* Lea el campo `texto_entrada` del estado.
* Convierte el texto a **minúsculas**.
* Elimina espacios al inicio y al final (`.strip()`).
* Devuelva el resultado en el campo `texto_salida`.



**Recuerda:** El nodo solo debe devolver un diccionario con los campos que modifica, no el estado completo.

In [ ]:
from typing import TypedDict

# TODO: 1. Define el estado del grafo

# TODO: 2. Implementa el nodo_transformar

# --- Bloque de Prueba ---
estado_test = {"texto_entrada": "   HOLA MUNDO   ", "texto_salida": ""}
resultado = nodo_transformar(estado_test)

if resultado:
    print(f"Resultado: '{resultado.get('texto_salida')}'")
    print("Esperado: 'hola mundo'")

---
## EJERCICIO 3: Pipeline de Procesamiento (Grafo Lineal)

**Paso 1:** Implementa 3 nodos con estas responsabilidades:
1. `nodo_validar`: Si el número es negativo, conviértelo a positivo (`abs()`)
2. `nodo_duplicar`: Multiplica el número por 2
3. `nodo_sumar_diez`: Suma 10 al número

**Paso 2:** Construye el grafo conectando los nodos linealmente:
```python
workflow.add_node("validar", nodo_validar)
workflow.set_entry_point("validar")
workflow.add_edge("validar", "duplicar")
workflow.add_edge("duplicar", "sumar_diez")
workflow.add_edge("sumar_diez", END)
app_numeros = workflow.compile()
```

**Resultado esperado:** `-5 → |-5| = 5 → 5*2 = 10 → 10+10 = 20`

In [ ]:
class EstadoNumero(TypedDict):
    numero: int

# TODO: Implementa los 3 nodos

# TODO: Construye el grafo

# Prueba con número negativo
estado_inicial = {"numero": -5}
resultado = app_numeros.invoke(estado_inicial)
print(f"Resultado: {resultado['numero']}")
print("Esperado: (|-5| * 2) + 10 = 20")

---
## EJERCICIO 4: Sistema de Decisión Dinámica (Router)

### Concepto: El Router como GPS Inteligente

Hasta ahora, nuestros agentes seguían un camino fijo. Pero los agentes reales necesitan tomar decisiones basándose en la información que tienen.

Un router es como un GPS: mira el estado actual y decide cuál es la mejor ruta a seguir.

En este ejercicio, construiremos un sistema de pricing que aplica descuentos diferentes según la categoría del producto.

### Instrucción Técnica

**Paso 1:** Implementa el router `router_precio` que categorice productos:
- Si `precio < 10` → devuelve `"barato"`
- Si `precio` entre 10 y 50 → devuelve `"medio"`
- Si `precio > 50` → devuelve `"caro"`

**Paso 2:** Implementa 3 nodos de descuento:
- `nodo_descuento_barato`: Aplica 5% descuento
- `nodo_descuento_medio`: Aplica 10% descuento
- `nodo_descuento_caro`: Aplica 15% descuento

**Paso 3:** Conecta todo con edges condicionales:
```python
workflow.add_conditional_edges(
    "categorizar",
    router_precio,
    {
        "barato": "nodo_descuento_barato",
        "medio": "nodo_descuento_medio",
        "caro": "nodo_descuento_caro"
    }
)
```

**Resultado:** El grafo automáticamente elige la ruta según el precio del producto.

In [ ]:
class EstadoProducto(TypedDict):
    precio: float
    categoria: str
    precio_final: float

# TODO: Implementa el nodo que categoriza
def nodo_categorizar(state: EstadoProducto) -> EstadoProducto:
    # Tu código aquí
    pass

# TODO: Implementa el router
def router_precio(state: EstadoProducto) -> str:
    # Tu código aquí
    pass

# TODO: Implementa los 3 nodos de descuento
def nodo_descuento_barato(state: EstadoProducto) -> EstadoProducto:
    # Aplicar 5% descuento
    pass

def nodo_descuento_medio(state: EstadoProducto) -> EstadoProducto:
    # Aplicar 10% descuento
    pass

def nodo_descuento_caro(state: EstadoProducto) -> EstadoProducto:
    # Aplicar 15% descuento
    pass

# TODO: Construye el grafo con edges condicionales
workflow_producto = StateGraph(EstadoProducto)
# ...

# Pruebas
productos_test = [
    {"precio": 5.0, "categoria": "", "precio_final": 0.0},
    {"precio": 30.0, "categoria": "", "precio_final": 0.0},
    {"precio": 100.0, "categoria": "", "precio_final": 0.0}
]

for producto in productos_test:
    resultado = app_producto.invoke(producto)
    print(f"Precio original: ${resultado['precio']:.2f} → Categoría: {resultado['categoria']} → Precio final: ${resultado['precio_final']:.2f}")

---
## EJERCICIO 5: Sistema de Validación Multi-Criterio

### Concepto: Verificaciones en Secuencia + Decisión Final

Muchos procesos del mundo real requieren múltiples verificaciones antes de tomar una decisión final. Por ejemplo, validar un email requiere verificar formato, dominio, y más.

Este patrón combina:
1. Nodos de verificación secuenciales (cada uno actualiza el estado con su resultado)
2. Router que evalúa TODOS los resultados para decidir la ruta final

### Instrucción Técnica

Construye un validador de emails con 2 verificaciones:

**Nodo 1 - `nodo_verificar_arroba`:**
- Verifica si el email contiene "@"
- Actualiza `tiene_arroba` a `True` o `False`

**Nodo 2 - `nodo_verificar_dominio`:**
- Verifica si termina en ".com", ".es" o ".org"
- Actualiza `dominio_valido` a `True` o `False`

**Router - `router_validacion`:**
- Si `tiene_arroba` Y `dominio_valido` ambos son `True` → `"valido"`
- Si no → `"invalido"`

**Nodos finales:** Uno confirma validez, otro rechaza el email.

In [ ]:
class EstadoEmail(TypedDict):
    email: str
    tiene_arroba: bool
    dominio_valido: bool
    resultado: str

# TODO: Implementa los nodos de verificación
def nodo_verificar_arroba(state: EstadoEmail) -> EstadoEmail:
    # Tu código aquí
    pass

def nodo_verificar_dominio(state: EstadoEmail) -> EstadoEmail:
    # Verificar si termina en .com, .es o .org
    pass

# TODO: Implementa el router
def router_validacion(state: EstadoEmail) -> str:
    # Si tiene_arroba Y dominio_valido → "valido", si no → "invalido"
    pass

# TODO: Implementa nodos finales
def nodo_email_valido(state: EstadoEmail) -> EstadoEmail:
    pass

def nodo_email_invalido(state: EstadoEmail) -> EstadoEmail:
    pass

# TODO: Construye el grafo completo
workflow_email = StateGraph(EstadoEmail)
# ...

# Pruebas
emails_test = [
    "usuario@example.com",
    "invalido.com",
    "test@dominio.es",
    "sin_arroba.org"
]

for email in emails_test:
    estado = {"email": email, "tiene_arroba": False, "dominio_valido": False, "resultado": ""}
    resultado = app_email.invoke(estado)
    print(f"Email: {email} → Resultado: {resultado['resultado']}")

---
## EJERCICIO 6: Razonamiento Iterativo (Ciclo con Límite)

### Concepto: Repetir Hasta Alcanzar un Objetivo

Los agentes avanzados necesitan iterar: repetir una acción hasta alcanzar una condición. Esto es lo que diferencia a LangGraph de LangChain.

Un ciclo es un edge que apunta hacia atrás a un nodo anterior, pero SIEMPRE debe tener una condición de salida.

En este ejercicio, crearás un contador que se incrementa iterativamente hasta alcanzar un límite configurable.

### Instrucción Técnica

**Nodo - `nodo_incrementar`:**
- Lee `contador` del estado
- Incrementa en 1
- Imprime el valor actual (para debugging)
- Devuelve el contador actualizado

**Router - `router_limite`:**
- Si `contador < limite` → devuelve `"continuar"` (CICLO: vuelve a incrementar)
- Si `contador >= limite` → devuelve `"terminar"` (salida del ciclo)

**Conexión del ciclo:**
```python
workflow.add_conditional_edges(
    "incrementar",
    router_limite,
    {
        "continuar": "incrementar",  # AQUÍ ESTÁ EL CICLO
        "terminar": "completar"
    }
)
```

**Resultado:** El nodo se ejecutará 5 veces (0→1→2→3→4→5) y luego terminará.

In [ ]:
class EstadoContadorConfigurable(TypedDict):
    contador: int
    limite: int
    mensaje: str

# TODO: Implementa el nodo incrementador
def nodo_incrementar(state: EstadoContadorConfigurable) -> EstadoContadorConfigurable:
    # Tu código aquí
    pass

# TODO: Implementa el router con ciclo
def router_limite(state: EstadoContadorConfigurable) -> str:
    # Si contador < limite → "continuar", si no → "terminar"
    pass

# TODO: Implementa nodo final
def nodo_completar(state: EstadoContadorConfigurable) -> EstadoContadorConfigurable:
    # Genera mensaje: "Contador completado: alcanzó X de X"
    pass

# TODO: Construye el grafo con ciclo
workflow_contador = StateGraph(EstadoContadorConfigurable)
# ...

# Prueba
estado_inicial = {"contador": 0, "limite": 5, "mensaje": ""}
resultado = app_contador.invoke(estado_inicial)
print(f"\n{resultado['mensaje']}")

# Visualiza la estructura del grafo
print("\n" + "="*60)
print("ESTRUCTURA DEL GRAFO:")
print("="*60)
print(app_contador.get_graph().draw_ascii())

---
## EJERCICIO 7: Lógica de Reintento (Retry con Máximo de Intentos)

### Concepto: Reintentar Hasta Éxito o Límite

Un patrón común en agentes reales es intentar una operación hasta que funcione, pero con un límite de intentos. Piensa en:
- Generación de contenido hasta que cumpla criterios de calidad
- Conexión a una API externa que puede fallar temporalmente
- Validación de inputs del usuario con feedback

Este patrón combina:
1. Ciclo de reintento (vuelve al nodo generador)
2. Dos condiciones de salida (éxito O máximo de intentos)

### Instrucción Técnica

Sistema de calificación con retry:

**Nodo - `nodo_generar_calificacion`:**
```python
calificacion = round(random.uniform(0, 10), 2)
intentos = state["intentos"] + 1
return {"calificacion": calificacion, "intentos": intentos}
```

**Router - `router_calificacion`:**
- Si `calificacion >= 5` → `"aprobado"` (ÉXITO)
- Si `intentos >= 3` → `"reprobado"` (LÍMITE ALCANZADO)
- Si no → `"reintentar"` (CICLO: vuelve a generar)

**Orden de prioridad:** Verifica primero si aprobó, LUEGO si llegó al límite.

**Resultado:** El agente reintentará hasta aprobar o hasta 3 intentos.

In [ ]:
import random

class EstadoCalificacion(TypedDict):
    calificacion: float
    intentos: int
    resultado: str

# TODO: Implementa el nodo que genera calificación aleatoria
def nodo_generar_calificacion(state: EstadoCalificacion) -> EstadoCalificacion:
    # Genera número aleatorio entre 0 y 10, incrementa intentos
    pass

# TODO: Implementa el router con lógica de retry
def router_calificacion(state: EstadoCalificacion) -> str:
    # Tu código aquí
    pass

# TODO: Implementa nodos finales
def nodo_aprobar(state: EstadoCalificacion) -> EstadoCalificacion:
    pass

def nodo_reprobar(state: EstadoCalificacion) -> EstadoCalificacion:
    pass

# TODO: Construye el grafo con ciclo
workflow_calificacion = StateGraph(EstadoCalificacion)
# ...

# Prueba (ejecuta varias veces para ver diferentes resultados)
estado_inicial = {"calificacion": 0.0, "intentos": 0, "resultado": ""}
resultado = app_calificacion.invoke(estado_inicial)
print(f"\nCalificación final: {resultado['calificacion']:.1f}")
print(f"Intentos necesarios: {resultado['intentos']}")
print(f"Resultado: {resultado['resultado']}")

---
## EJERCICIO 8: Pipeline de Negocio Multi-Etapa

### Concepto: Proceso de Negocio con Validación + Transformación + Decisión

Los procesos empresariales reales combinan múltiples patrones:
- Validación temprana (bifurcación según disponibilidad)
- Transformaciones secuenciales (cálculos)
- Lógica condicional (aplicar descuentos según umbral)

Este ejercicio integra todo lo aprendido en un flujo de procesamiento de pedidos.

### Instrucción Técnica

**Flujo del agente:**

1. **`nodo_validar_stock`**: Verifica si hay inventario suficiente
2. **`router_stock`**: 
   - Si `cantidad_pedido <= stock_disponible` → `"procesar"`
   - Si no → `"cancelar"`
3. **Pipeline de procesamiento** (solo si hay stock):
   - `nodo_calcular_total`: `total = cantidad * precio_unitario`
   - `nodo_aplicar_descuento`: Si `total > 100` → aplica 10% descuento
   - `nodo_confirmar`: Marca pedido como CONFIRMADO
4. **Ruta de cancelación:**
   - `nodo_cancelar`: Marca pedido como CANCELADO

**Resultado:** Dos rutas posibles según disponibilidad de stock.

In [ ]:
class EstadoPedido(TypedDict):
    cantidad_pedido: int
    stock_disponible: int
    precio_unitario: float
    total: float
    estado_pedido: str

# TODO: Implementa los nodos del pipeline
def nodo_validar_stock(state: EstadoPedido) -> EstadoPedido:
    # Solo imprime, no modifica estado (el router decide)
    pass

def router_stock(state: EstadoPedido) -> str:
    # "procesar" o "cancelar"
    pass

def nodo_calcular_total(state: EstadoPedido) -> EstadoPedido:
    # total = cantidad * precio
    pass

def nodo_aplicar_descuento(state: EstadoPedido) -> EstadoPedido:
    # Si total > 100, aplica 10% descuento
    pass

def nodo_confirmar(state: EstadoPedido) -> EstadoPedido:
    # estado_pedido = "CONFIRMADO"
    pass

def nodo_cancelar(state: EstadoPedido) -> EstadoPedido:
    # estado_pedido = "CANCELADO"
    pass

# TODO: Construye el grafo completo
workflow_pedido = StateGraph(EstadoPedido)
# ...

# Pruebas
pedido_con_stock = {
    "cantidad_pedido": 5,
    "stock_disponible": 10,
    "precio_unitario": 25.0,
    "total": 0.0,
    "estado_pedido": ""
}

pedido_sin_stock = {
    "cantidad_pedido": 15,
    "stock_disponible": 10,
    "precio_unitario": 25.0,
    "total": 0.0,
    "estado_pedido": ""
}

print("PEDIDO 1 (Con stock):")
resultado1 = app_pedido.invoke(pedido_con_stock)
print(f"Estado: {resultado1['estado_pedido']} - Total: ${resultado1['total']:.2f}\n")

print("PEDIDO 2 (Sin stock):")
resultado2 = app_pedido.invoke(pedido_sin_stock)
print(f"Estado: {resultado2['estado_pedido']}")

---
## EJERCICIO 9: Sistema de Generación + Validación Iterativa

### Concepto: Generar Hasta Cumplir Criterios de Calidad

Un patrón común en agentes generativos es:
1. Generar un output (texto, código, contraseña, etc.)
2. Validar que cumple los criterios
3. Si no cumple → regenerar
4. Repetir hasta éxito o límite de intentos

Este es el patrón usado por sistemas como correctores de código, validadores de contenido, etc.

### Instrucción Técnica

**Sistema de generación de contraseñas seguras:**

**Nodo 1 - `nodo_generar_password`:**
```python
import string
caracteres = string.ascii_letters + string.digits
password = ''.join(random.choices(caracteres, k=8))
intentos = state["intentos"] + 1
return {"password": password, "intentos": intentos}
```

**Nodo 2 - `nodo_validar_password`:**
- Verifica que tenga al menos 1 letra Y 1 número
- Actualiza `es_valida` a `True` o `False`

**Router - `router_password`:**
- Si `es_valida` → `"aceptar"`
- Si `intentos >= 5` → `"fallo"`
- Si no → `"regenerar"` (CICLO: vuelve a generar)

**Resultado:** El agente generará passwords hasta encontrar una válida o agotar intentos.

In [ ]:
import string
import random

class EstadoPassword(TypedDict):
    password: str
    es_valida: bool
    intentos: int
    resultado: str

# TODO: Implementa el generador
def nodo_generar_password(state: EstadoPassword) -> EstadoPassword:
    # Genera password aleatoria de 8 caracteres
    pass

# TODO: Implementa el validador
def nodo_validar_password(state: EstadoPassword) -> EstadoPassword:
    # Verifica al menos 1 letra Y 1 número
    pass

# TODO: Implementa el router
def router_password(state: EstadoPassword) -> str:
    # "aceptar", "fallo" o "regenerar"
    pass

# TODO: Implementa nodos finales
def nodo_aceptar_password(state: EstadoPassword) -> EstadoPassword:
    pass

def nodo_fallo_password(state: EstadoPassword) -> EstadoPassword:
    pass

# TODO: Construye el grafo
workflow_password = StateGraph(EstadoPassword)
# ...

# Prueba
estado_inicial = {"password": "", "es_valida": False, "intentos": 0, "resultado": ""}
resultado = app_password.invoke(estado_inicial)
print(f"\nPassword generada: {resultado['password']}")
print(f"Intentos necesarios: {resultado['intentos']}")
print(f"Resultado: {resultado['resultado']}")

---
## EJERCICIO 10: Sistema Multi-Agente con Bifurcaciones Anidadas

### Concepto: Arquitectura de Decisión en Cascada

Los sistemas complejos requieren múltiples niveles de decisión. Cada decisión puede desencadenar nuevas bifurcaciones.

Este ejercicio final integra TODOS los patrones:
- Validaciones secuenciales
- Múltiples routers anidados
- Rutas de éxito y rechazo en cada etapa

Es el tipo de arquitectura que verías en un sistema de aprobación de créditos, revisión de documentos legales, o diagnóstico médico.

### Instrucción Técnica

**Sistema de aprobación de crédito con 3 etapas:**

**Etapa 1 - Verificación de edad:**
- `nodo_verificar_edad`: No hace nada, solo lee
- `router_edad`: Si `edad >= 18` → `"verificar_ingresos"`, si no → `"rechazar_edad"`

**Etapa 2 - Verificación de ingresos:**
- `nodo_verificar_ingresos`: No hace nada
- `router_ingresos`: Si `ingresos_anuales >= 20000` → `"verificar_score"`, si no → `"rechazar_ingresos"`

**Etapa 3 - Score crediticio:**
- `nodo_verificar_score`: No hace nada
- `router_score`: 
  - Si `score >= 700` → `"aprobar_premium"` (5% interés)
  - Si `score >= 600` → `"aprobar_standard"` (10% interés)
  - Si no → `"rechazar_score"`

**Nodos finales:** 6 salidas posibles (3 rechazos + 2 aprobaciones).

**Resultado:** Un sistema de decisión con 3 niveles de bifurcación.

In [ ]:
class EstadoCredito(TypedDict):
    edad: int
    ingresos_anuales: float
    score_crediticio: int
    decision: str
    tasa_interes: float

# TODO: Implementa los nodos de verificación (solo leen, no modifican)
def nodo_verificar_edad(state: EstadoCredito) -> EstadoCredito:
    return state

def nodo_verificar_ingresos(state: EstadoCredito) -> EstadoCredito:
    return state

def nodo_verificar_score(state: EstadoCredito) -> EstadoCredito:
    return state

# TODO: Implementa los 3 routers
def router_edad(state: EstadoCredito) -> str:
    # "verificar_ingresos" o "rechazar_edad"
    pass

def router_ingresos(state: EstadoCredito) -> str:
    # "verificar_score" o "rechazar_ingresos"
    pass

def router_score(state: EstadoCredito) -> str:
    # "aprobar_premium", "aprobar_standard" o "rechazar_score"
    pass

# TODO: Implementa los 6 nodos finales
def nodo_rechazar_edad(state: EstadoCredito) -> EstadoCredito:
    pass

def nodo_rechazar_ingresos(state: EstadoCredito) -> EstadoCredito:
    pass

def nodo_rechazar_score(state: EstadoCredito) -> EstadoCredito:
    pass

def nodo_aprobar_premium(state: EstadoCredito) -> EstadoCredito:
    # decision="APROBADO - Premium", tasa_interes=5.0
    pass

def nodo_aprobar_standard(state: EstadoCredito) -> EstadoCredito:
    # decision="APROBADO - Standard", tasa_interes=10.0
    pass

# TODO: Construye el grafo con 3 niveles de bifurcación
workflow_credito = StateGraph(EstadoCredito)
# ...

# Pruebas
casos_test = [
    {"edad": 25, "ingresos_anuales": 50000, "score_crediticio": 750, "decision": "", "tasa_interes": 0.0},
    {"edad": 25, "ingresos_anuales": 30000, "score_crediticio": 650, "decision": "", "tasa_interes": 0.0},
    {"edad": 17, "ingresos_anuales": 40000, "score_crediticio": 700, "decision": "", "tasa_interes": 0.0},
]

for i, caso in enumerate(casos_test, 1):
    resultado = app_credito.invoke(caso)
    print(f"\nCaso {i}: Edad={caso['edad']}, Ingresos=${caso['ingresos_anuales']}, Score={caso['score_crediticio']}")
    print(f"Decisión: {resultado['decision']}")
    if resultado['tasa_interes'] > 0:
        print(f"Tasa de interés: {resultado['tasa_interes']}%")